In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import TensorBoard
import time

%config Completer.use_jedi = False 

In [3]:
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.333)
sess = tf.compat.v1.Session(config = tf.compat.v1.ConfigProto(gpu_options=gpu_options))

### Load Data

In [4]:
X = np.load("X.npy")
y = np.load("y.npy")

### Normalize features.

In [5]:
X = tf.keras.utils.normalize(X)

## `Model`

In [10]:
NAME = "{}-conv-{}-nodes-{}-dense-{}".format(3, 128, 1, int(time.time()))
tensorboard = TensorBoard(log_dir='bcd_3logs')
print(NAME)

model = Sequential()

model.add(Conv2D(128, (3,3), input_shape = X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(128, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))


model.add(Dense(1))
model.add(Activation('sigmoid'))

opt = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

model.fit(X, y, batch_size=24, epochs=12, validation_split=0.3, callbacks=[tensorboard])



3-conv-128-nodes-1-dense-1618844665
Epoch 1/12
100/100 [==============================] - 58s 584ms/step - loss: 0.6404 - accuracy: 0.6884 - val_loss: 0.5866 - val_accuracy: 0.7316
Epoch 2/12
100/100 [==============================] - 58s 575ms/step - loss: 0.5984 - accuracy: 0.7127 - val_loss: 0.5694 - val_accuracy: 0.7375
Epoch 3/12
100/100 [==============================] - 58s 580ms/step - loss: 0.5934 - accuracy: 0.7119 - val_loss: 0.5610 - val_accuracy: 0.7424
Epoch 4/12
100/100 [==============================] - 59s 589ms/step - loss: 0.5885 - accuracy: 0.7161 - val_loss: 0.5776 - val_accuracy: 0.7218
Epoch 5/12
100/100 [==============================] - 58s 580ms/step - loss: 0.5708 - accuracy: 0.7148 - val_loss: 0.5448 - val_accuracy: 0.7316
Epoch 6/12
100/100 [==============================] - 59s 590ms/step - loss: 0.5616 - accuracy: 0.7228 - val_loss: 0.5498 - val_accuracy: 0.7307
Epoch 7/12
100/100 [==============================] - 59s 587ms/step - loss: 0.5476 - accuracy

### Saving

In [11]:
model.save('bcd_model.h5')

### Testing


In [17]:
import cv2
import tensorflow as tf

CATEGORIES = ["Cancer", "Normal"]

def prepare(filepath):
    IMG_SIZE = 100
    img_arr = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_arr = cv2.resize(img_arr, (IMG_SIZE, IMG_SIZE))
    return new_arr.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

model = tf.keras.models.load_model('bcd_model.h5')

In [18]:
prediction = model.predict([prepare('test_samples/Cancer/c_2.png')])
print(CATEGORIES[int(prediction[0][0])])
print(int(prediction[0][0]))

Cancer
0


In [19]:
prediction = model.predict([prepare('test_samples/Normal/n_1.png')])
print(CATEGORIES[int(prediction[0][0])])
print(int(prediction[0][0]))

Normal
1
